Check Bash Version
------------------------------

In [ ]:
echo $BASH_VERSION

Variables
--------------

In [ ]:
myname="Hans Schmid"
echo "Hello, $myname"

In [ ]:
echo $myname

String interpolation

In [ ]:
echo "$myname"

Literal string

In [ ]:
echo '$MYNAME'

In [ ]:
type pwd

In [ ]:
echo $(pwd)

In [ ]:
echo $PWD

Backticks are regarded as obsolete. But hard to kill in the field.

In [ ]:
echo `pwd`

Keywords
---------------
Braces, double brackets and exclamation mark are keywords. That means in order to use them correctly you need spaces around them!

In [ ]:
type if then elif else fi time for in until while do done case esac coproc select function { } [[ ]] !

Equal sign is not a keyword. So when using it you do not put spaces around it.

In [ ]:
type =

which is obsolete. You'll find it a lot though.
<br>
**'which' is a bitch.**

In [ ]:
type which

Use **type -p** instead because it's a builtin!

In [ ]:
type -p which

In [ ]:
type type

# Functions

1. Functions in Bash are not functions like in other languages. They are actually commands. Functions are used as if they were command line binaries or scripts.
2. Shell commands are connected by pipes (aka streams), and not by fundamental or user-defined data types as in _real_ programming languages. There is no such thing as a return value for a command. (Although there is a return statement - see below.)
3. When a function wants to get input it reads it from its input stream, or the argument list. In both cases text strings have to be parsed. The shell with its pipe-based architecture is purely string-based!
4. When a command wants to return something it has to _echo/printf_ it to its output stream.
5. The caller can use command substitution _$()_ to capture the output.
6. Another practiced way is to store the return values in dedicated, global variables. (Writing to the output stream is clearer and more flexible, because it can also take binary data.)
7. Think of the exit code as a bonus that other languages don't have, or as a "Schmutzeffekt" of shell functions. The meaning of the exit code is up to the shell programmer!
8. Often by convention an exit status of 0 means success, 1-255 means failure and the exit code can be regarded as a failure number.
9. Just to make sure: _return/exit_ can only take a value from 0-255. Values other than 0 are _not necessarily_ errors. There is always an exit status.
10. The argument list is only for decoration and you never put anything inside them.

## How to define functions
There are two ways to define functions.

### 1. With the **_function_** keyword:
a) The argument list is optional.<br>
b) Coding style: Do not use an argument list.

In [ ]:
function success {
    echo "I am successful!"
    return 0
}

### 2. Without the **_function_** keyword.
The argument list is mandatory.

In [ ]:
success() {
    echo "I am successful!"
    return 0
}

### Passing and Accessing Arguments

In [ ]:
function printme {
    echo "You gave me $# argument(s)!"
    echo -n $1
}

In [ ]:
printme

In [ ]:
printme 'Hello'

In [ ]:
printme 'Hello' 'World'

### Returning Values

#### 1. echo/printf values to the output stream

In [ ]:
function add {
    # $((...)) to calculate mathematical expressions
    sum=$(($1+$2))
    echo -n $sum
}

In [ ]:
add 1 2

#### 1. Capture Return Values via Command Substition Using $()

In [ ]:
result=$(add 1 2)
echo $result

#### 2. Use Global Variables to Provide Return Values
The _sum_ variable in our add function is actually a global variable.

In [ ]:
echo -n $sum

### Variable Scope
Using local variables in order to not pollute the global namespace.

In [ ]:
function add_local {
    local sum=$(($1+$2))
    echo -n $sum
}

In [ ]:
add_local 5 6

The global variable from above did not change.

In [ ]:
echo -n $sum

#### 3. Argument References
As of Bash 4.3+, we can pass an input argument by reference.

In [ ]:
function add_by_ref {
    declare -n add_ref=$3
    add_ref=$(($1+$2)) 
}

In [ ]:
add_by_ref 1 2 myAddRefResult

In [ ]:
echo -n $myAddRefResult

### Sub-shells
1. A sub-shell is a special type of command group that allows us to spawn a new execution environment from the current shell.
2. Instead of curly braces, we use parentheses to delimit the function body.

In [ ]:
function add_subshell (
    # 'sum' is a global variable
    sum=$(($1+$2))
    echo -n $sum 
)

In [ ]:
add_subshell 2 3

Our global variable did not change because the function was executed in a sub-shell.

In [ ]:
echo -n $sum

### Recursion
Recursion is possible but not typical for shell programming.

In [ ]:
function factorial {
    if [ $1 -le 1 ]; then
        echo -n 1
    else
        echo -n $(($(factorial $(($1-1)))*$1))
    fi 
}

In [ ]:
factorial 5

In [ ]:
factorial 10

# Arrays

## Declaring Arrays
There are two ways:<br>
1. Declare a variable explicitly to be an array.
2. Create an array on the fly.

#### Declaring an array explicitly

In [ ]:
declare -p pioneers

In [ ]:
declare -a pioneers

In [ ]:
declare -p pioneers

#### Creating an Array on the Fly
An array is automatically created when you assign a value to a variable.

In [ ]:
pioneers[0]='Ken Thompson'

In [ ]:
declare -p pioneers

You can assign multiple values at once.

In [ ]:
unset pioneers

In [ ]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie')

In [ ]:
declare -p pioneers

#### Assigning Values to an Array
1. An array doesn't need to have continuous indexes.
2. When you try to print an array element that is not initialized, you'll get a null value.

In [ ]:
pioneers[20]='Douglas McIlroy'

In [ ]:
declare -p pioneers

### Accessing Array Elements

The default index is zero.

In [ ]:
echo $pioneers

This does not work as one might expect coming from a mainstream programming language.

In [ ]:
echo $pioneers[1]

This is the correct way.

In [ ]:
echo ${pioneers[1]}

Now the whole array:

In [ ]:
echo ${pioneers[@]}

In [ ]:
echo ${pioneers[*]}

We"ll see the difference of the two commands above when we cover looping.

In [ ]:
declare -p pioneers

### Appending an Array Element

In [ ]:
unset pioneers

You can also explicitly specify the index of the array entry.

In [ ]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie' [20]='Douglas McIlroy')

In [ ]:
pioneers+=('Linus Torvals' 'Richard Stallman')

In [ ]:
declare -p pioneers

You often see this workaround in older scripts:

In [ ]:
unset pioneers

In [ ]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie' [20]="Douglas McIlroy")

In [ ]:
declare -p pioneers

This workaround of appending array elements works only when indices are consecutive.

In [ ]:
pioneers=("${pioneers[@]}" 'Joe Ossanna' 'Linus Torvalds')

In [ ]:
declare -p pioneers

The quotes are essential!

In [ ]:
pioneers=(${pioneers[@]} 'Richard Stallman')

In [ ]:
declare -p pioneers

### Slicing Arrays

In [ ]:
unset pioneers

Without using quotes.

In [ ]:
pioneers=(Ken Brian Dennis Douglas)

First parameter is index, second parameter is length!
Note the parens!

In [ ]:
pioneers2=(${pioneers[@]:1:2})

In [ ]:
declare -p pioneers2

Not what you expected:

In [ ]:
pioneers2=${pioneers[@]:1:2}

In [ ]:
declare -p pioneers2

Slice from the beginning:

In [ ]:
pioneers2=(${pioneers[@]::2})

In [ ]:
declare -p pioneers2

Slice from the end:

In [ ]:
pioneers2=(${pioneers[@]:2})

In [ ]:
declare -p pioneers2

### Getting Indices

In [ ]:
pioneers=(Ken Brian Dennis Douglas)

In [ ]:
echo -n ${!pioneers[@]}

In [ ]:
pioneers=(Ken Brian Dennis Douglas [10]=Joe)

In [ ]:
echo -n ${!pioneers[@]}

### Removing Array Elements

In [36]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie' [20]="Douglas McIlroy")

: 1

Bye, bye, Brian!

In [ ]:
unset pioneers[1]

In [ ]:
declare -p pioneers

### Array Length

In [ ]:
echo -n ${#pioneers[@]}

In [ ]:
echo -n ${#pioneers[*]}

In [ ]:
echo -n 'Length of the third element:' ${#pioneers[2]}

### Looping through an Array

In [ ]:
i=1
for item in "${pioneers[@]}"; do
    echo $((i++)). $item
done

In [34]:
i=1
for item in "${pioneers[*]}"; do
    echo $((i++)). $item
done

1. Ken Thompson Brian Kernighan Dennis Ritchie Douglas McIlroy


: 1

In [45]:
for index in ${!pioneers[@]}; do
    echo -e 'Index: '$index'\t'${pioneers[$index]}
done

Index: 0	Ken Thompson04l
Index: 1	Brian Kernighan
Index: 2	Dennis Ritchie
Index: 20	Douglas McIlroy


: 1

#### Using File Globbing to Initialize an Array

In [ ]:
files=(/etc/[abcdefg]*.conf)

In [ ]:
declare -p files

In [ ]:
i=1
for file in ${files[@]}; do
    echo $((i++)). $file
done

#### Loading File Content into an Array

In [ ]:
unset pioneers

In [ ]:
cat pioneers.txt

In [ ]:
pioneers=( $(cat pioneers.txt) )

In [ ]:
declare -p pioneers

In [ ]:
for pioneer in ${pioneers[@]}; do
    echo $pioneer
done

#### Loading Arrays from a File with _mapfile_

In [28]:
mapfile -t pioneers < pioneers.txt # -t strips newline

: 1

In [29]:
declare -p pioneers

declare -a pioneers=([0]="Ken" [1]="Dennis" [2]="Brian" [3]="Douglas" [4]="Joe")


: 1

You can leave out an array name. Bash will use MAPFILE in this case.

In [30]:
mapfile -t < pioneers.txt

: 1

In [31]:
declare -p MAPFILE

declare -a MAPFILE=([0]="Ken" [1]="Dennis" [2]="Brian" [3]="Douglas" [4]="Joe")


: 1

# Associate Arrays

## Differences from Arrays
1. declare -A (seems to be mandatory!)
2. Index is a string, not a number (0,1,2,3,..)

In [3]:
declare -A proglangs=([Python]="Guido van Rossum" [Ruby]="Yukihiro Matsumoto" [C++]="Bjarne Stroustrup")

: 1

In [4]:
declare -p proglangs

declare -A proglangs=([Python]="Guido van Rossum" [Ruby]="Yukihiro Matsumoto" [C++]="Bjarne Stroustrup" )


: 1

In [5]:
echo -n ${!proglangs[@]}

Python Ruby C++

: 1

In [6]:
echo ${proglangs[Python]}

Guido van Rossum


: 1

In [8]:
proglangs[Erlang]="Joe Armstrong"

: 1

In [9]:
declare -p proglangs

declare -A proglangs=([Python]="Guido van Rossum" [Ruby]="Yukihiro Matsumoto" [Erlang]="Joe Armstrong" [C++]="Bjarne Stroustrup" )


: 1

In [10]:
proglangs+=([Javascript]="Brendan Eich" [Perl]="Larry Wall")

: 1

In [11]:
declare -p proglangs

declare -A proglangs=([Python]="Guido van Rossum" [Ruby]="Yukihiro Matsumoto" [Perl]="Larry Wall" [Erlang]="Joe Armstrong" [Javascript]="Brendan Eich" [C++]="Bjarne Stroustrup" )


: 1

In [12]:
echo -n ${!proglangs[@]}

Python Ruby Perl Erlang Javascript C++

: 1

In [13]:
echo -n ${#proglangs[@]}

6

: 1

In [14]:
unset proglangs[Javascript]

: 1

In [15]:
echo -n ${#proglangs[@]}

5

: 1

In [16]:
echo -n ${!proglangs[@]}

Python Ruby Perl Erlang C++

: 1

In [18]:
i=1
for proglang in "${proglangs[@]}"; do
    echo $((i++)). $proglang
done

1. Guido van Rossum2004l
2. Yukihiro Matsumoto
3. Larry Wall
4. Joe Armstrong
5. Bjarne Stroustrup


: 1

In [31]:
for key in ${!proglangs[@]}; do
    echo -e $key'\t'${proglangs[$i]}
done

Python	Bjarne Stroustrup
Ruby	Bjarne Stroustrup
Perl	Bjarne Stroustrup
Erlang	Bjarne Stroustrup
C++	Bjarne Stroustrup


: 1

# Loops
Looping constructs in Bash:
1. For Loops
2. For Loops/C-Syntax
3. While Loops
4. Until Loops
5. Case Command
6. Select Command

## For Loops
General structure:
<br><code>
for varname in list
do
    commands using $varname"
done
</code>

In [37]:
for day in Mon Tue Wed Thu Fri; do
    echo "Weekday: $day"
done

Weekday: Mon004l
Weekday: Tue
Weekday: Wed
Weekday: Thu
Weekday: Fri


: 1

In [38]:
weekdays="Mon Tue Wed Thu Fri"
for day in $weekdays; do
    echo "Weekday: $day"
done

Weekday: Mon004l
Weekday: Tue
Weekday: Wed
Weekday: Thu
Weekday: Fri


: 1

But not:

In [39]:
weekdays="Mon Tue Wed Thu Fri"
for day in "$weekdays"; do
    echo "Weekday: $day"
done

Weekday: Mon Tue Wed Thu Fri004l


: 1

In [41]:
for username in $(awk -F: '{print $1}' /etc/passwd); do
    echo "Username: $username"
done

Username: root4l
Username: nobody
Username: _uuidd
Username: schmidh
Username: _dhcpcd
Username: polkitd
Username: dbus
Username: sddm
Username: _mlocate
Username: rtkit
Username: pulse
Username: ldap
Username: tss
Username: openntpd
Username: cups
Username: transmission
Username: tor
Username: mpd


: 1

In [43]:
for item in /tmp/*; do
    echo "$item"
done

/tmp/Temp-488512d8-c936-4b6c-9585-fc7446339a9e
/tmp/Temp-4c203723-8531-4c8d-9642-0ad508721129
/tmp/dropbox-antifreeze-MrxsjD
/tmp/dropbox-antifreeze-beccw3
/tmp/plasma-csd-generator
/tmp/sddm-auth67d4c07b-5760-489a-b242-c56354fe58f4
/tmp/xauth-1000-_0


: 1

In [44]:
for item in /tmp/d*; do
    echo "$item"
done

/tmp/dropbox-antifreeze-MrxsjD
/tmp/dropbox-antifreeze-beccw3


: 1

#### Breaking out of a Loop

In [47]:
for day in Mon Tue Wed Thu Fri; do
    if [ $day == "Thu" ]; then
        break;
    fi
    echo "Weekday: $day"
done

Weekday: Mon004l
Weekday: Tue
Weekday: Wed


: 1

#### Continuing a Loop

In [49]:
for day in Mon Tue Wed Thu Fri Sat Sun; do
    echo -n "$day"
    if [ $day == "Sat" -o $day == "Sun" ]; then
        echo " (WEEKEND)"
        continue;
    fi
    echo " (weekday)"
done

Mon (weekday)04l
Tue (weekday)
Wed (weekday)
Thu (weekday)
Fri (weekday)
Sat (WEEKEND)
Sun (WEEKEND)


: 1

#### Range of numbers

In [51]:
for num in {1..10}; do
    echo "Number: $num"
done

Number: 1[?2004l
Number: 2
Number: 3
Number: 4
Number: 5


: 1

In [52]:
for num in {1..10..2}; do
    echo "Number: $num"
done

Number: 1[?2004l
Number: 3
Number: 5
Number: 7
Number: 9


: 1

## For Loops Using C-Syntax

In [56]:
for (( i=1; i <= 5; i++ )); do
    echo "Random number $i: $((RANDOM%100))"
done

Random number 1: 502004l
Random number 2: 36
Random number 3: 87
Random number 4: 32
Random number 5: 37


: 1

Interrupt kernel when you're getting bored:

In [59]:
for (( ; ; )); do
    sleep $((RANDOM%10))
    echo "Number: $((i++))"
done

Number: 15?2004l
Number: 16
Number: 17
Number: 18
Number: 19
Number: 20
Number: 21
Number: 22



In [62]:
for ((i=1, j=10; i <= 5 ; i++, j=j+5)); do
    echo "Number $i: $j"
done

Number 1: 10004l
Number 2: 15
Number 3: 20
Number 4: 25
Number 5: 30


: 1

## While Loops

In [63]:
type true

true is a shell builtin


: 1

In [71]:
i=1
while [ $i -le 5 ]; do
  echo "$((i++)) time(s)"
done

1 time(s)[?2004l
2 time(s)
3 time(s)
4 time(s)
5 time(s)


: 1

## Until Loops

In [72]:
i=1
until [ $i -gt 5 ]; do
    echo "$((i++)) time(s)."
done

1 time(s).?2004l
2 time(s).
3 time(s).
4 time(s).
5 time(s).


: 1

## Case Command

General structure:
<code>
case  $variable-name  in
      pattern1|pattern2|pattern3)       
        command1
        ...
        ....
        commandN
        ;;
      pattern4|pattern5|pattern6)
     	command1
        ...
        ....
        commandN
        ;;            
      pattern7|pattern8|patternN)       
     	command1
        ...
        ....
        commandN
        ;;
      *)              
esac
</code>



In [81]:
NOW=$(date +"%a")
case $NOW in
    Mon)
        echo "FULL backup!";;
    Tue|Wed|Thu|Fri)
        echo "PARTIAL backup!";;
    Sat|Sun)
        echo "NO backup!";;
    *) ;;
esac

PARTIAL backup!l


: 1

### Case terminators

The ;& terminator causes case to also execute the next block without testing its pattern. The ;;& operator is like ;; except the case statement doesn't terminate after executing the associated list. Bash just continues testing the next pattern as though the previous pattern didn't match.

## Select Command

Execute this in a real Bash shell:

<code>select state in yes no dk; do
    echo "Your selection is: $state"
    break # without it you have an endless loop
done</code>